# 第5章：语言模型

语言模型（language model，LM）在自然语言处理中占有重要的地位，尤其在基于统计模型的语音识别、机器翻译、汉语自动分词和句法分析等相关研究中得到了广泛应用。

目前主要采用的是$n$元语法模型（$n$-gram model），这种模型构建简单、直接，但同时也因为数据缺乏而必须采取平滑（smoothing）算法。

## 5.1 $n$元语法

一个语言模型通常构建为字符串$s$的概率分布$p(s)$，$p(s)$表示字符串$s$作为一个句子出现的频率。*与语言学中不同，语言模型与句子是否合乎语法是没有关系的，即使一个句子完全合乎语法逻辑，我们仍然可以认为它出现的概率接近为零。*

一个由$l$个基元（“基元”可以为字、词或短语等，一般用“词”来通指）构成的句子$s = w_{1} w_{2} \cdots w_{l}$，其概率计算公式可以表示为

$$\begin{aligned}
p(s) & = p(w_{1}) p(w_{2} | w_{1})p(w_{3} | w_{1} w_{w}) \cdots p(w_{l} | w_{1} \cdots w_{l - 1}) \\
& = \prod_{i = 1}^{l} p(w_{i} | w_{1} \cdots w_{i - 1})
\end{aligned} \tag {5-1}$$

方程（5-1）中，第$i$（$1 \leq i \leq l$）个词出现的概率是由已出现的$i - 1$个词$w_{1} w_{2} \cdots w_{i - 1}$决定的。一般地前$i - 1$个词$w_{1} w_{2} \cdots w_{i - 1}$称为第$i$个词的“历史（history）”。随历史长度$i - 1$的增加，不同的历史数目呈指数级增长（$L^{i - 1}$，假设$L$为词汇集的大小）。考虑在所有$L^{i - 1}$种不同的历史情况下，产生第$i$个词的概率，模型中有$L^{i - 1}$个自由参数$p(w_{i} | w_{1} \cdots w_{i - 1})$。 假设$L = 5000$、$i = 3$，则自由参数数量为1250亿。若将历史$w_{1} w_{2} \cdots w_{i - 1}$按照某个法则映射到等价类$E(w_{1} w_{2} \cdots w_{i - 1})$，而等价类的数目远远小于不同历史的数目，即假定：

$$p(w_{i} | w_{1} \cdots w_{i - 1}) = p\left( w_{i} | E(w_{1} w_{2} \cdots w_{i - 1}) \right) \tag {5-2}$$

则自由参数数量会极大减少。

**$n$元语法（$n$元文法，$n$-gram）语言模型**：将两个历史$w_{i - n + 2} \cdots w_{i - 1} w_{i}$和$v_{k - n + 2} \cdots v_{k - 1} v_{k}$映射到同一个等价类，当且仅当这两个历史最近的$n - 1$（$1 \leq n \leq l$）个词相同，即如果$E \left( w_{1} w_{2} \cdots w_{i - 1} w_{i} \right) = E \left( v_{1} v_{2} \cdots v_{k - 1} v_{k} \right)$，当且仅当$w_{i - n + 2} \cdots w_{i - 1} w_{i} = v_{k - n + 2} \cdots v_{k - 1} v_{k}$，通常$n = 3$。

$n = 1$，即出现在第$i$位上的词$w_{i}$独立于历史，称为一元文法（unigram、uni-gram、monogram）；

$n = 2$，即出现在第$i$位上的词$w_{i}$仅与它前面的一个历史词$w_{i - 1}$有关，称为二元文法（bigram、bi-gram）模型，或一阶马尔可夫链（Markov chain）；

$n = 3$，即出现在第$i$位上的词$w_{i}$仅与它前面的两个历史词$w_{i - 2} w_{i - 1}$有关，称为三元文法（trigram、tri-gram）模型，或二阶马尔可夫链。

在二元语法模型中，一个词的概率只依赖于它前面的一个词，则

$$\begin{aligned}
p(s) & = \prod_{i = 1}^{l} p(w_{i} | w_{1} \cdots w_{i - 1}) \approx \prod_{i = 1}^{l} p(w_{i} | w_{i - 1})
\end{aligned} \tag {5-3}$$

为使$p(w_{i} | w_{i - 1})$对于$i = 1$有意义，在句子开头加句首标记`<BOS>`，即假设$w_{0}$为`<BOS>`。为使所有的字符串的概率之和$\sum_{s} p(s)$等于1，需在句尾添加句尾标记`<EOS>`，并且使之包含在方程（5-3）的乘积中（否则，所有给定长度的字符串的概率和为1，所有字符串的概率和为无穷大)。

例：计算概率$p(\text{Mark wrote a book})$：

$$p(\text{Mark wrote a book}) = p(\text{Mark} | \text{<BOS>}) \times p(\text{wrote} | \text{Mark}) \times p(\text{a} | \text{wrote}) \times p(\text{book} | \text{a}) \times p(\text{<EOS>}| \text{book})$$

用于构建语言模型的文本称为训练语料（training corpus），对于$n$元语法模型，使用的训练语料的规模一般为百万级别。


二元语法条件概率$p(w_{i} | w_{i - 1})$的估计为计算$w_{i - 1} w_{i}$在语料库中出现的频率，然后归一化：

$$p(w_{i} | w_{i - 1}) = \frac{c(w_{i - 1} w_{i})}{\sum_{w_{i}} c(w_{i - 1} w_{i})} \tag {5-4}$$

其中，$c(w_{i - 1} w_{i})$表示二元语法$w_{i - 1} w_{i}$在给定语料库中的出现次数。方程（5-4）中估计$p(w_{i} | w_{i - 1})$的方法称为$p(w_{i} | w_{i - 1})$的最大似然估计（maximum likelihood estimation, MLE）。对于$n \gt 2$的$n$元语法模型，条件概率中要考虑前面$n - 1$个词的概率，方程（5-3）改写为：

$$\begin{aligned}
p(s) & = \prod_{i = 1}^{l + 1} p(w_{i} | w_{i - n + 1}^{i - 1})
\end{aligned} \tag {5-5}$$

其中，$w_{i}^{j}$表示词$w_{i} \cdots w_{j}$，约定$w_{- n + 2}$到$w_{0}$为`<BOS>`，取$w_{l + 1}$为`<EOS>`。$p(w_{i} | w_{i - n + 1}^{i - 1})$的最大似然估计为：

$$p(w_{i} | w_{i - n + 1}^{i - 1}) = \frac{c(w_{i - n + 1}^{i})}{\sum_{w_{i}} c(w_{i - n + 1}^{i})} \tag {5-6}$$

<img src="./img/page_85_1.jpg" width="500" />

P.S.：方程（5-4）为$p(w_{i} | w_{i - 1})$的最大似然估计

已知$w_{i - 1}$出现的条件下，下一词为$w_{i}$的概率服从伯努利分布，即$p(w_{i} | w_{i - 1}) \sim $



In [1]:
# bigram

def process_sent(sent):
    sent = "<BOS> " + sent.strip().lower() + " <EOS>"
    return sent.split(" ")

def process_corpus(corpus):
    corpus_split = []
    vocab = set()
    for sent in corpus:
        sent = process_sent(sent)
        corpus_split.append(sent)
        vocab.update(sent)
        
    return corpus_split, vocab

def calc_word_count(corpus, vocab):
    word_count = {word: 0 for word in vocab}
    for sent in corpus:
        for word in vocab:
            word_count[word] += sent.count(word)
            
    return word_count

def calc_word_count_bigram(corpus):
    bigram = {}
    for sent in corpus:
        for i in range(len(sent) - 1):
            word_pair = sent[i], sent[i + 1]
            if bigram.get(word_pair):
                bigram[word_pair] += 1
            else:
                bigram[word_pair] = 1
                
    return bigram

def calc_sent_prob_bigram(sent, word_count, word_count_bigram):
    prob = 1
    for i in range(len(sent) - 1):
        word_pair = sent[i], sent[i + 1]
        prob *= word_count_bigram[word_pair] / word_count[word_pair[0]]
        
    return prob

In [2]:
corpus = ["BROWN READ HOLY BIBLE", "MARK READ A TEXT BOOK", "HE READ A BOOK BY DAVID"]

corpus, vocab = process_corpus(corpus)
word_count = calc_word_count(corpus, vocab)
word_count_bigram = calc_word_count_bigram(corpus)

sent = "BROWN READ A BOOK"
sent = process_sent(sent)
prob = calc_sent_prob_bigram(sent, word_count, word_count_bigram)

print(prob)

0.05555555555555555


## 5.2 语言模型性能评价

评价语言模型常用度量是模型计算的测试数据的概率，或利用交叉熵（crossentropy）和困惑度（perplexity）等派生测度。对于一个平滑过的概率为$p(w_{i} | w_{i - n + 1}^{i - 1})$的$n$元语法模型，用方程（5-5）计算句子的概率$p(s)$。

给定由句子$t_{1}, t_{2}, \cdots, t_{l_{T}}$构成的测试集$T$，通过计算$T$中所有句子概率的乘积来计算测试集的概率$p(T)$：

$$p(T) = \prod_{i = 1}^{l_{T}} p(t_{i})$$

交叉熵测度可根据预测和压缩的关系计算。给定一个语言模型，文本$T$的概率为$p(T)$，给定一个压缩算法，该算法用$\log_{2} p(T)$个比特位来对文本$T$编码，在数据$T$上模型$p(w_{i} | w_{i - n + 1}^{i - 1})$的交叉熵$H_{p}(T)$定义为

$$H_{p}(T) = - \frac{1}{W_{T}} \log_{2} p(T) \tag {5-7}$$

其中，$W_{T}$是以词为单位度量的文本$T$的长度（可以包括句首标志`<BOS>`或句尾标志`<EOS>`）。方程（5-7）表示利用与模型$p(w_{i} | w_{i - n + 1}^{i - 1})$有关的压缩算法对数据集合中的$W_{T}$个词进行编码，每一个编码所需要的平均比特位数。

模型$p$的困惑度$\text{PP}_{T}(T)$是模型分配给测试集$T$中每一个词汇的概率的几何平均值的倒数，它和交叉熵的关系为

$$\text{PP}_{T}(T) = 2^{H_{p}(T)}$$

交叉熵和困惑度越小越好，这是评估一个语言模型的基本准则。在英语文本中，$n$元语法模型计算的困惑度范围大约为$50 \sim 1000$之间（对应的交叉熵范围为$6 \sim 10$个比特位）具体值与文本的类型有关。


## 5.3 数据平滑

### 5.3.1 问题的提出

根据给定的训练语料$S$计算句子概率时，如果句子$s$中出现训练语料中未包含的$n$元语法$w_{i - n + 1}^{i}$，由方程（5-6）可知$p(w_{i} | w_{i - n + 1}^{i - 1}) = 0$，即$p(s) = 0$。显然，这个结果并不合理，因为句子$s$总有出现的可能，其概率应大于0。

语音识别的任务目标为：给定声音信号$A$，找到转写句子$s$，使概率$p(s | A) = \frac{p(A | s)p(s)}{p(A)}$最大。如果$p(s) = 0$，则$p(s | A)$必然为0，即不管给定的语音信号多么清晰，字符串$s$永远不可能成为转写结果。因此，在语音识别中，一旦出现使得$p(s) = 0$的字符串$s$，就会导致识别错误。在其他自然语言处理任务中也会出现类似的问题。因此，必须分配给所有可能出现的字符串一个非零的概率值来避免这种错误的发生。

**平滑**（smoothing）技术是用来解决这类零概率问题的。术语“平滑”指的是为了产生更准确的概率（在方程（5-4）和方程（5-6）中）来调整最大似然估计的一种技术，也常称为数据平滑（data smoothing）。“平滑”处理的基本思想是“劫富济贫”，即提高低概率（如零概率），降低高概率，尽量使概率分布趋于均匀。

加1法：对于二元语法，假设每个二元语法出现的次数比实际出现的次数多一，即

$$p(w_{i} | w_{i - 1}) = \frac{1 + c(w_{i - 1} w_{i})}{\sum_{w_{i}} [1 + c(w_{i - 1} w_{i})]} = \frac{1 + c(w_{i - 1} w_{i})}{ |V| + \sum_{w_{i}} c(w_{i - 1} w_{i})} \tag {5-8}$$

其中，$V$表示所有词汇的单词表，$|V|$表示词汇表单词的个数。词汇表总是有限的，可以大约固定在几万个或者几十万个。所有不在词汇表中的词可以映射为一个单个的区别于其他已知词汇的单词，通常将其称为**未登录词**或**未知词**（out of vocabulary，OOV）。

<img src="./img/page_87_1.jpg" width="650" />

In [13]:
# bigram plus one

def calc_sent_prob_bigram_plus_one(sent, word_count, word_count_bigram):
    prob = 1
    vocab_size = len(word_count) - 2 # <BOS> and <EOS> are not counted in
    for i in range(len(sent) - 1):
        word_pair = sent[i], sent[i + 1]
        prob *= (1 + word_count_bigram.get(word_pair, 0)) / (vocab_size + word_count.get(word_pair[0], 0))
        
    return prob

In [14]:
sent = "BROWN READ A BOOK"
sent = process_sent(sent)
prob = calc_sent_prob_bigram_plus_one(sent, word_count, word_count_bigram)

print(prob)

sent = "DAVID READ A BOOK"
sent = process_sent(sent)
prob = calc_sent_prob_bigram_plus_one(sent, word_count, word_count_bigram)

print(prob)

0.00012075836251660427
3.0189590629151068e-05


In [12]:
len(word_count)

13

### 5.3.2 加法平滑方法

实际应用中最简单的平滑技术之一是**加法平滑方法**（additive smoothing）[G.J.Lidstone, W.E.Johnson and H.Jeffreys]：假设每一个$n$元语法发生的次数比实际统计次数多发生$\delta$（$0 \leq \delta \leq 1$）次，则，

$$p(w_{i} | w_{i - 1}) = \frac{\delta + c(w_{i - 1} w_{i})}{ \delta|V| + \sum_{w_{i}} c(w_{i - 1} w_{i})} \tag {5-8}$$


G.J. Lidstone和H. Jeffreys曾提倡取$\delta = 1$，但有些学者认为这种方法一般表现较差。

### 5.3.3 古德-图灵（Good-Turing）估计法

Good-Turing估计法[I.J. Good, 1953]是很多平滑技术的核心：对于任何一个出现$r$次的$n$元语法，都假设它出现了$r^{\ast}$次，

$$r^{\ast} = (r + 1) \frac{n_{r + 1}}{n_{r}} \tag {5-10}$$

其中，$n_{r}$是训练语料中恰好出现$r$次的$n$元语法的数目。通过归一化，将该统计数转为概率：统计数为$r$的$n$元语法，其概率为

$$p_{r} = \frac{r^{\ast}}{N} \tag {5-11}$$

其中，$N = \sum_{r = 0}^{\infty} n_{r} r^{\ast}$。

$$N = \sum_{r = 0}^{\infty} n_{r} r^{\ast} = \sum_{r = 0}^{\infty} (r + 1)  n_{r + 1} = \sum_{r = 1}^{\infty} n_{r} r \tag {5-A}$$

即，$N$等于该分布中最初的计数，样本中所有事件的概率之和为

$$\sum_{r \gt 0} n_{r} p_{r} = 1 - \frac{n_{1}}{N} \lt 1$$

因此，可以将$\frac{n_{1}}{N}$的概率剩余量分配给所有未见事件（$r = 0$的事件）。

<img src="./img/page_88_1.jpg" width="650" />

通常情况下，Good-Turing方法作为一个基本方法，应用于其它平滑方法。Good-Turing方法不能直接用于估计$n_{r} = 0$的$n$-gram概率，此外，
Good-Turing方法不能实现高阶模型与低阶模型的结合，而高低阶模型的结合通常是获得较好的平滑效果所必须的。

In [17]:
import numpy as np

In [32]:
r = list(range(1, 8))
n_r = [2053, 458, 191, 107, 69, 48, 36]

r_ast = [None] * len(r)

for idx in range(len(r) - 1):
    r_ast[idx] = (r[idx] + 1) * n_r[idx + 1] / n_r[idx]

N = np.sum([r[idx] * n_r[idx] for idx in range(len(r))])
p_r = [r_ast[idx] / N if r_ast[idx] else None for idx in range(len(r))]

print("r_ast:", r_ast)
print("N:", N)
print("p_r:", p_r)


r_ast: [0.4461763273258646, 1.2510917030567685, 2.2408376963350785, 3.2242990654205608, 4.173913043478261, 5.25, None]
N: 4855
p_r: [9.190037638019868e-05, 0.000257691390948871, 0.00046155256361175666, 0.0006641192719712793, 0.0008597143240943895, 0.0010813594232749742, None]


### 5.3.4 Katz平滑方法

Katz平滑方法[Katz, 1987]通过加入高阶模型与低阶模型的结合，对Good-Turing估计方法进行扩展。

二元语法模型的Katz平滑方法：对于一个出现次数为$r = c(w_{i - 1}^{i})$的二元语法$w_{i - 1}^{i}$，修正计数为：

$$c_{\text{katz}}(w_{i - 1}^{i}) = \begin{cases}
d_{r} r, & r \gt 0 \\
a(w_{i - 1}) p_{\text{ML}}(w_{i}), & r = 0
\end{cases} \tag {5-12}$$

即，所有非零计数$r$的二元语法都根据折扣率$d$减值，折扣率$d_{r}$近似等于$\frac{r^{\ast}}{r}$，该减值是由Good-Turing估计方法预测的。从非零计数中减去的计数量，根据低一阶的分布，即一元语法模型，被分配给计数为零的二元语法。通过选择$a(w_{i - 1})$的值，使分布中总的计数$\sum_{w_{i}} c_{\text{katz}}(w_{i - 1}^{i})$保持不变，即$\sum_{w_{i}} c_{\text{katz}}(w_{i - 1}^{i}) = \sum_{w_{i}} c(w_{i - 1}^{i})$
a(w, 1)的适当值为
1- 2 Pun(w; |w-r) 1-2 Ppan(w; | w-)
w;:c(w-1)>0
w:e(w-1)>0
a(w;-1) = -
2 PML (w;)
1- 2 PM.(we,) 
w:c(wi_ 1)=0
q;c(w21)>0
要根据修正的计数计算概率Par(w;|w;-1),只需要归一化:
phatz(w;| w-l) =一
Ckatz(w}_ 1)
2ckanu(w/-) .
折扣率d,可以按照如下办法计算:由于大的计数值是可靠的，因此它们不需要减
值。尤其对于某些k,S. M. Katz取所有r>k情况下的d,=1,并且建议k=5。对于
r≤k情况下的折扣率,减值率由用于全局二元语法分布的Good-Turing估计方法计算，
即公式(5-10)中的n,表示在训练语料中恰好出现r次的二元语法的总数。d,的选择遵
循如下约束条件:①最终折扣量与Good-Truing估计预测的减值量成比例;②全局二元
语法分布中被折扣的计数总量等于根据Good-Turing估计应该分配给次数为零的二元
语法的总数。第一个约束条件相当于对于某些常数μ,r∈{1,2,..,k}有公式:
1-d,=u(1-5)
